In [ ]:
! pip install pyreadstat

In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
import numpy as np # linear algebra


In [ ]:
#pd.set_option('display.max_rows', 100)

In [ ]:
Ovin = pd.read_spss("../input/ovin2014/OViN2014_Databestand.sav")

The dataset is from OVin 2014 Dutch human activity survey. The dataset took quite sometime to preprocess. There are lots of categorical data and missing values. Also the digits are separated with comma, but it is harder to replace because they are in the text. The functions below show some basic info. The first challenge is to extract useful info, e.g. numerical values from the text. The "not run" block shows some inital efforts. 

In [ ]:
Ovin.describe()
Ovin.info()
column_names = Ovin.columns
for column in column_names:
    print(column + ' - ' + str(Ovin[column].isnull().sum()))

In [ ]:
Ovin['Rvm_num'] = pd.Categorical(Ovin.Rvm).astype('category').codes.tolist()
Ovin['KAfstR_num'] = pd.Categorical(Ovin.KAfstR).astype('category').codes.tolist()

In [ ]:
### not run

#These are some first tries but not useful in the end. 

#Ovin[["Rvm", "KAfstR"]].groupby([ "KAfstR"]).describe(include='all')
#a = Ovin.apply(pd.Series.replace, to_replace ="tot", value =".") only replace the entire string instead of a character

# The attempt to replace "," and extract values. After many tries I came to the for loop. It was 1 o'clock at night...
'''
naive method
for i in range(len(Ovin["KAfstR"])):
    if   type(Ovin.loc[i,"KAfstR"]) is str:
           Ovin["numericKa"] = Ovin.loc[i,"KAfstR"].replace(",",".")
Ovin["numericKa"]= Ovin["numericKa"].str.slice(stop = 3).astype(int)
'''



The functions below are the final solution. Still interating over each cell of a column, then extract values or take some statistics. THe the new values are added in the dataframe. 

In [ ]:
def extract_d(string):
    string = re.sub(",", ".", string)
    a = re.findall( r"[-+]?\d*\.\d+|\d+", string) 
    if a:
        return a

def lst_mean(lst): 
    return sum(lst) / len(lst) 
 

In [ ]:
#not a1[38] is np.nan or a1[38] is None

In [ ]:
def add_extracted(Orig, old_col, new_col, toextract):
    a2 = np.empty(len(Orig))
    a = Orig[old_col]
    #.dropna()
    #a = a.reset_index(drop=True) 
    a1 = a.apply(lambda x: extract_d(x))
    
    for i in range(len(a1)):
        if a1[i] is np.nan or a1[i] is None:
             continue
        afl = [round(float(j),1) for j in a1[i]] # string in list to float
        if toextract is "mean":
            a2[i]= int(lst_mean(afl))
        elif toextract is "last":
            a2[i] = afl[len(afl)-1]
        elif type(toextract) is int:   
            a2[i]= afl[toextract]
        
    Orig[new_col] = a2.tolist()
    return Orig    

# example: Ovin = add_extracted(Ovin,"KAfstR",'KAf_low', 0 )

Plotting the counts of one variables grouped by another

In [ ]:
def plot_rel_bar(ps, filterfreq, savename,ifstack = True):
    freq_filtered =  ps[ps>filterfreq] #filter out freq. less than 1, see a more sophisticated method below that group them into "others"
    plt.figure(figsize=(20, 10), dpi=100, facecolor='w', edgecolor='k')
    #plt.subplots(1,2)
    freq_filtered.unstack().plot(kind='bar',stacked = ifstack, cmap="tab20")
    plt.legend(loc= 'upper left',bbox_to_anchor=(-1, 1), ncol=1) 
    plt.show() 
    plt.savefig(savename,bbox_inches='tight')

In [ ]:
# Journey travel time: KAfstR Ritafstandklasse
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
Ovin["KAfstR"].value_counts(sort = False).plot(kind = 'bar')

In [ ]:
# To analyse the text, extracting the lower and upper boundary and calculate the mean travel time from the text. 
Ovin = add_extracted(Ovin,"KAfstR",'KAf_low', 0 )
Ovin = add_extracted(Ovin,"KAfstR",'KAf_mean', "mean" )
Ovin = add_extracted(Ovin,"KAfstR",'KAf_high', "last" )

In [ ]:
Ovin[["KAfstR","KAf_mean", "KAf_low","KAf_high"]].head(3)

In [ ]:
# journey mode: RVM Ritvervoermiddel
figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
Ovin["Rvm"].value_counts().plot(kind = 'bar')

In [ ]:
a = Ovin.query('KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Rvm","KAf_low"]].groupby("KAf_low")["Rvm"].value_counts(normalize = True)
plot_rel_bar(a, 0.1, "RvmKAfstR.png")


## different activities
Now let's have a look at the relationships in different activities.

In [ ]:
# activities we have
Ovin["Doel"].unique().tolist()

In [ ]:
Ovin["Doel"].value_counts(normalize = True).plot(kind = 'bar')

### For going to work
Most people are close to their home but actually a lot more peole travel further than 15 km.
Most people take the car with distance > 5. This is interestingly matching with common sense. For 0.5 m there are still people go by bike. But someone go by car! There is noone working closer than 500 m. It is also interesting that longer than 50 m people turned to go by car.

In [ ]:
Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)["KAfstR"].value_counts(normalize=True)

In [ ]:
Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)["KAfstR"].value_counts().plot(kind = 'bar')

In [ ]:
# for going to work, most people take the car with distance > 5. This is interestingly matching with common sense. For 0.5 m there are still people go by bike. But there is noone working closer than 500 m
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Rvm","KAf_low"]].groupby("KAf_low")["Rvm"].value_counts(normalize = False)
plot_rel_bar(a, 20, "work_RvmKAfstR.png") # filter if less than 20 people
#a

In [ ]:
# for going to work, most people take the car with distance > 5. This is interestingly matching with common sense. For 0.5 m there are still people go by bike. But there is noone working closer than 500 m
pd.set_option('display.max_rows', 200)
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Rvm","KAf_low"]].groupby("KAf_low")["Rvm"].value_counts(normalize = True)
plot_rel_bar(a, 0.1, "work_RvmKAfstR.png")
a = round(a [a>0.05], 1)
pd.Series.to_csv(a, "NL_distance_travelmode2work.csv")

In [ ]:
student= Ovin["MaatsPart"].unique().tolist()[1]
 
# for going to work, most people take the car with distance > 5. This is interestingly matching with common sense. For 0.5 m there are still people go by bike. But there is noone working closer than 500 m
a = Ovin[Ovin["MaatsPart"] == student].query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Rvm","KAf_low"]].groupby("KAf_low")["Rvm"].value_counts(normalize = True)
plot_rel_bar(a, 0.1, "work_RvmKAfstR.png")
a =round(a [a>0.05], 1)
pd.Series.to_csv(a, "student_distance_travelmode.csv")

## Departure time
Let's get the travel departure time, the lower and upper boundary.

In [ ]:
#KVertTijd: Vertrektijdklasse
Ovin=add_extracted(Ovin,"KVertTijd",'KVTearly', 0 ) 
Ovin=add_extracted(Ovin,"KVertTijd",'KVlate', "last" ) 

In [ ]:
# most people leave early to work. quite some at 4 ??
Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KVTearly", ascending=True)["KVTearly"].value_counts().plot(kind = 'bar')

In [ ]:
# doesnt seem to have strong relationshio with travel time
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["KVTearly","KAf_low"]].groupby("KVTearly")["KAf_low"].value_counts(normalize = False)
plot_rel_bar(a, 30, "KVTearlyKAfstR.png")  


In [ ]:
# doesnt seem to have strong relationshio with travel time
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["KVTearly","KAf_low"]].groupby("KVTearly")["KAf_low"].value_counts(normalize = False)
plot_rel_bar(a, 30, "KVTearlyKAfstR.png")  


In [ ]:
# time departing vs. distance
Ovin.query('Doel=="Toeren/wandelen" & KAf_low>0.1')[["KAf_mean", "KVTearly"]].corr()
Ovin.query('Doel=="Werken" & KAf_low>0.1')[["KAf_mean", "KVTearly"]].corr()

In [ ]:
# BHvm : travel mode for businuess, but only 1185 records
Ovin["BHvm"].dropna() # only  1185
Ovin["BHvm"].value_counts(normalize = True).plot(kind = 'bar')

### travel Frequency 

In [ ]:
Ovin["OVGebruik"].value_counts(normalize = True).plot(kind = 'bar')

In [ ]:
Ovin["OVStKaart"].value_counts(normalize = False).plot(kind = 'bar')

In [ ]:
Ovin["HHFiets"].value_counts(normalize = False).plot(kind = 'bar')
Ovin["Brandstof"].value_counts(normalize = False).plot(kind = 'bar')


###Income

In [ ]:
Ovin=add_extracted(Ovin,"HHGestInk",'Income_lb', 0 ) 

In [ ]:
Ovin["HHGestInk"].value_counts(normalize = False).plot(kind = 'bar')
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Income_lb","KAf_low"]].groupby("Income_lb")["KAf_low"].value_counts(normalize = False)
plot_rel_bar(a, 30, "KVTearlyKAfstR.png")  
 

In [ ]:
a = Ovin.query('Doel=="Werken" & KAf_low>0.1').sort_values(by="KAf_low", ascending=False)[["Income_lb","Rvm"]].groupby("Income_lb")["Rvm"].value_counts(normalize = False)
plot_rel_bar(a, 30, "werk_inc_Rvn.png")  
 

In [ ]:
#rvm Ritvervoermiddel
a = Ovin.query('Doel=="Werken" & KAf_low>0.1 & Income_lb>30.0 ').sort_values(by="KAf_low", ascending=False)[["Income_lb","Rvm"]].groupby("Income_lb")["Rvm"].value_counts(normalize = False)
plot_rel_bar(a, 30, "werk_inc_Rvn.png", False)  

a = Ovin.query('Doel=="Werken" & KAf_low>0.1 & Income_lb<=30.0 ').sort_values(by="KAf_low", ascending=False)[["Income_lb","Rvm"]].groupby("Income_lb")["Rvm"].value_counts(normalize = False)
plot_rel_bar(a, 30, "werk_inc_Rvn.png", False)  

In [ ]:
# hvm Hoofdvervoermiddel verplaatsing

a = Ovin.query('Doel=="Werken" & KAf_low>0.1 & Income_lb>30.0 ').sort_values(by="KAf_low", ascending=False)[["Income_lb","Hvm"]].groupby("Income_lb")["Hvm"].value_counts(normalize = False)
plot_rel_bar(a, 30, "werk_inc_Rvn.png", False)  

a = Ovin.query('Doel=="Werken" & KAf_low>0.1 & Income_lb<=30.0 ').sort_values(by="KAf_low", ascending=False)[["Income_lb","Hvm"]].groupby("Income_lb")["Hvm"].value_counts(normalize = False)
plot_rel_bar(a, 30, "werk_inc_Rvn.png", False)  
 

In [ ]:
Ovin[["KAf_low", "Income_lb", "KVTearly","Rvm_num"]].corr()

In [ ]:
Ovin=add_extracted(Ovin,"KLeeft",'age_lb', 0 ) 

In [ ]:
Ovin["age_lb"].value_counts(normalize = False).plot(kind = 'bar')
a = Ovin.query('Doel=="Werken" & KAf_low>0.1' ).sort_values(by="age_lb", ascending=False)[["age_lb","KAf_low"]].groupby("age_lb")["KAf_low"].value_counts(normalize = False)
plot_rel_bar(a, 30, "KVTearlyKAfstR.png")  
# we seepeole younger than 18 and older than 65 dont travel very far, but besides there seems to have no pattern

In [ ]:
Ovin[["KAf_low", "Income_lb", "KVTearly","age_lb"]].corr()

### Sptial content
"Sted" urban or not. It is strange that there seem to be no relationship between travel distance and urban or not. 

In [ ]:
sns.set_palette("pastel")

g=sns.catplot(x="Sted", y="Income_lb", jitter = False, hue='Rvm', data=Ovin.query('Doel=="Werken" & KAf_low>0.1&Income_lb>10000' ))
g.set_xticklabels(rotation=30)


In [ ]:
a = Ovin.query('Doel=="Werken" & KAf_low>0.1' ).sort_values(by="age_lb", ascending=False)[["Sted","Income_lb"]].groupby("Sted")["Income_lb"].value_counts(normalize = False)
plot_rel_bar(a, 30, "income_urbanornot.png") 

In [ ]:
a = Ovin.query('Doel=="Werken" & KAf_low>0.1' ).sort_values(by="age_lb", ascending=False)[["Sted","KAf_low"]].groupby("Sted")["KAf_low"].value_counts(normalize = False)
plot_rel_bar(a, 30, "distance_urbanornot.png") 

In [ ]:
Ovin.to_csv("Ovin.csv",index=False)

In [ ]:
Ovin=add_extracted(Ovin,"KVertTijd",'KVlate', "last" ) 

In [ ]:
'''
for i in range(len(a1)):
    if a1[i] != []:
        a2.append(int(float(a1[i][0])))
        
'''

In [ ]:
''' merge small frequencies https://milos.ai/2019/01/merging-categories-with-small-frequencies
threshold_percent = 1

series = pd.value_counts(df['company_location'])
mask = (series / series.sum() * 100).lt(threshold_percent)
df = df.assign(company_location_updated = np.where(df['company_location'].isin(series[mask].index),'Other', df['company_location']))
'''
